In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Xarray

expande las capacidades de manipular datos de los arrays de Numpy 
- Domain agnostic
- Dimensions
- Metadata
- Open source

<img src="images/xarray.png" width="1024" height="512">


## Data structures

<img src="images/data_structures.png"  width="1280" height="300">

**Data array** guarda variables individuales multi-dimensionales, y sus coordenadas.
(alt: wrap underlying containers (e.g. numpy arrays) and contain associated metadata)

**Data set** guarda variables múltiples que potencialmente comparten la misma coordenada


## DataSet

## DataSet

Especie de diccionario que contiene varios DataArrays y metadata.
- **Dimensiones** diccionario con [nombre de dimension]:[tamaño fijo]
- **Coordenadas** diccionario que contiene DataArrays del mismo tipo que data_vars.
    - (DataArray, variable, tuplas, objeto de pandas,...)
- **Data variables** variables extra que me pueden servir, por ejemplo acá _air_ se define como un array de (time, lat, lon)
- **Atributos** metadata extra

#### Convención
*Coordenadas* indican cantidades constantes/fijas/independientes , en cambio las cantidades variantes/dependientes pertenecen a las *Data variables*.



In [ ]:
ds = xr.open_dataset("./air_temperature.nc")
display(ds)

## "Numpy" en Xarray

Las funciones universales de numpy también funcionan para cualquier objeto de Xarray, y devuelve algo de tipo Xarray.


In [ ]:
ds.lat

In [ ]:
new = np.cos(np.deg2rad(new))
new.rename("lat modified")

## Broadcasting

Multiplicando dos DataArray de 1D obtienes un DataArray de 2D
➡️ Xarray automatiza broadcast

<img src="images/broadcasting.png" width="864" height="648">

## Indexing con Xarray


Una vez tengamos nuestro data array queremos "aprovechar" nuestras coordenas, y por algun motivo no perder la posibilidad de ser infelices:
Para ello podemos aprovecharlos y seleccionar a traves indices (porque wtf?) o a traves de nombres dimension:


In [30]:
example = xr.DataArray(
    np.arange(0, 60).reshape(3, 4, 5),
    coords={
        "dim1": ["A", "B", "C"],
        "dim2": ["a", "b", "c", "d"],
        "dim3": np.arange(0, 5)
    },
    dims=("dim1", "dim2", "dim3")
)

In [26]:
example[2,1,2]

<xarray.DataArray ()> Size: 8B
array(47)
Coordinates:
    dim1     <U1 4B 'C'
    dim2     <U1 4B 'b'
    dim3     int64 8B 2

In [24]:
example.loc["C","b",2]

<xarray.DataArray ()> Size: 8B
array(47)
Coordinates:
    dim1     <U1 4B 'C'
    dim2     <U1 4B 'b'
    dim3     int64 8B 2

In [19]:
example.sel(dim1 = slice("A","C"))

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 5)> Size: 480B
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])
Coordinates:
  * dim1     (dim1) <U1 12B 'A' 'B' 'C'
  * dim2     (dim2) <U1 16B 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 40B 0 1 2 3 4

In [ ]:
example.isel(dim1 = 0)

In [33]:
example.drop_sel(dim3 = [2,3])

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 3)> Size: 288B
array([[[ 0,  1,  4],
        [ 5,  6,  9],
        [10, 11, 14],
        [15, 16, 19]],

       [[20, 21, 24],
        [25, 26, 29],
        [30, 31, 34],
        [35, 36, 39]],

       [[40, 41, 44],
        [45, 46, 49],
        [50, 51, 54],
        [55, 56, 59]]])
Coordinates:
  * dim1     (dim1) <U1 12B 'A' 'B' 'C'
  * dim2     (dim2) <U1 16B 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 24B 0 1 4

In [ ]:
example2 = example.assign_coords(dim1 = [97.8,98.2,344.23123])
example2

In [ ]:
example2.sel(dim1 = 97)

# DateTime Indexing:

Podes tener una dimension que sus coordenas sean fechas, entonces logicamente deberia andar bien el seleccionar una fecha puntual.. anda bien :D, pero no solo eso sino que dado que esta guardado en formato fecha podes hacer cosas como:
| Objetivo                                                            | Código `xarray`                                                             | Tipo             |
| ------------------------------------------------------------------- | --------------------------------------------------------------------------- | ---------------- |
| Fecha exacta (día y hora)                                           | `ds.sel(time='2013-01-01 06:00')`                                           | Exacta           |
| Día completo (todas las horas)                                      | `ds.sel(time='2013-01-01')`                                                 | Exacta           |
| Mes completo (todas las fechas de ese mes y año)                    | `ds.sel(time='2014-05')`                                                    | Exacta           |
| Año completo                                                        | `ds.sel(time='2013')`                                                       | Exacta           |
| Slice entre dos fechas                                              | `ds.sel(time=slice('2013-01-01', '2013-01-10'))`                            | Slice            |
| Slice entre dos fechas con hora                                     | `ds.sel(time=slice('2013-01-01 03:00', '2013-01-02 12:00'))`                | Slice            |
| Slice de mes a mes                                                  | `ds.sel(time=slice('2013-01', '2013-03'))`                                  | Slice            |
| Slice de año a año                                                  | `ds.sel(time=slice('2012', '2014'))`                                        | Slice            |
| Todos los valores de un **mes** sin importar año                    | `ds.sel(time=ds.time.dt.month == 7)`                                        | Condicional      |
| Todos los valores de un **día del mes** sin importar mes/año        | `ds.sel(time=ds.time.dt.day == 15)`                                         | Condicional      |
| Todos los valores de un **año específico**                          | `ds.sel(time=ds.time.dt.year == 2014)`                                      | Condicional      |
| Todos los valores de una **hora específica**                        | `ds.sel(time=ds.time.dt.hour == 6)`                                         | Condicional      |
| Todos los valores de un **día de la semana (0=lunes)**              | `ds.sel(time=ds.time.dt.dayofweek == 0)`                                    | Condicional      |
| Todos los valores de una **fecha específica combinada (mes + día)** | `ds.sel(time=(ds.time.dt.month == 12) & (ds.time.dt.day == 24))`            | Combinada        |
| Estación: Verano (dic-ene-feb)                                      | `ds.sel(time=ds.time.dt.month.isin([12, 1, 2]))`                            | Estacional       |
| Estación: Otoño (mar-abr-may)                                       | `ds.sel(time=ds.time.dt.month.isin([3, 4, 5]))`                             | Estacional       |
| Estación: Invierno (jun-jul-ago)                                    | `ds.sel(time=ds.time.dt.month.isin([6, 7, 8]))`                             | Estacional       |
| Estación: Primavera (sep-oct-nov)                                   | `ds.sel(time=ds.time.dt.month.isin([9, 10, 11]))`                           | Estacional       |
| Estación combinada con año (ej: invierno 2013)                      | `ds.sel(time=(ds.time.dt.year == 2013) & ds.time.dt.month.isin([6, 7, 8]))` | Estacional + Año |
| Días impares                                                        | `ds.sel(time=ds.time.dt.day % 2 == 1)`                                      | Personalizada    |
| Último día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_end)`                                      | Especial         |
| Primer día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_start)`                                    | Especial         |


## COMPUTATION
Probablemente querramos hacer cosas mas interesantes que preguntar por datos en nuestro xarray, asi que veamos que podemos hacer :D
# BASIC
podemos aplicar funciones y hacer cualquier reduccion estandar de numpy en las coordenadas (mean, min, max, sum, etc.)

In [38]:
example = xr.DataArray(
    np.zeros((3,4)),
    coords = {
        "x" : np.arange(0,3),
        "y" : np.arange(0,4)
    },
    dims = ("x","y")
)
example

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [39]:
def sumar(a,b):
    return a+b

xr.apply_ufunc(sumar,example,2)

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [42]:
from math import *
(example + 2) ** 2

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [53]:
arr = np.array([
    [0, 1, 2, 3],
    [8, 9, 10, 11],
    [0, 5, 6, 7]
])
example.data = arr
example

<xarray.DataArray (x: 3, y: 4)> Size: 96B
array([[ 0,  1,  2,  3],
       [ 9,  9, 10, 11],
       [ 0,  5,  6,  7]])
Coordinates:
  * x        (x) int64 24B 0 1 2
  * y        (y) int64 32B 0 1 2 3

In [55]:
example.mean(dim = "x")

<xarray.DataArray (y: 4)> Size: 32B
array([3., 5., 6., 7.])
Coordinates:
  * y        (y) int64 32B 0 1 2 3

# Computing with multiple objects


## ¿Porqué X array?

La **metadata** hace el código más legible    
➡️ reduce la posibilidad de errores y typos, y hace el análisis maś divertido!

ejemplo:
```py
lat = ds.air.lat.data  # numpy array
lon = ds.air.lon.data  # numpy array
temp = ds.air.data  # numpy array
```
### Sin Xarray 😟
```py
plt.figure()
plt.pcolormesh(lon, lat, temp[0, :, :]);

```

### Con Xarray 😊
```py
ds.air.mean(dim="time").plot(x="lon")
```

## Dask 👻
buuu computación paralelaaa